### Pre-requisites
1. Install [Ollama](https://ollama.com/download)
2. In the command prompt: run `ollama pull llama3.2` and optionally `ollama serve`
3. git clone the project
4. Create a conda virtual environment and `pip install -r requirements.txt`
5. Place [qdrant]() folder in the project folder
7. Place [peft-sent-model]() folder in `src/backend` folder
8. Place [peft-sent-model]() folder in the Project folder (This is for the jupyter notebook)

### Install Qdrant
1. Install [Docker](https://docs.docker.com/desktop/setup/install/windows-install/)
2. In the project folder, run : `docker pull qdrant/qdrant`
3. The run `docker run -p 6333:6333 -p 6334:6334 -v ./qdrant_storage:/qdrant/storage:z qdrant/qdrant`

### For the UI / backend code (Not Jupyter)
1. Do the steps above (from 1 to 6) and Qdrant installation
2. Execute `uvicorn financeengine:app --host 0.0.0.0 --port 8025` in `src/backend` folder (in another terminal)
3. Execute `streamlit run financechatclient.py` in `src/frontend` folder (in another terminal)
4. Open `http://localhost:8501/`

### Imports
These are mainly transformers, llama_index for vector store, pandas, qdrant etc

In [35]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import uuid
import json
import os
import datetime
from tqdm import tqdm
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, Document, StorageContext
import torch
from llama_index.llms.ollama import Ollama
from pydantic import BaseModel
from llama_index.core.node_parser import SentenceSplitter
from datasets import load_dataset
from IPython.display import display, HTML
import abc
from typing import List, Dict
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from transformers import Trainer
from transformers import Trainer
from torch.utils.data import Subset
import requests
from fastapi import FastAPI
from peft import PeftModel
from transformers import AutoModelForSeq2SeqLM
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from llama_index.core.base.llms.types import ChatMessage, ChatResponse, MessageRole
from transformers import AutoTokenizer, GenerationConfig

#### Run the below if you dont have nltk punkt (uncomment) 


In [6]:
# import nltk
# nltk.download('punkt')

### The below code sets the environment and llm used for RAG

1. We first set the base model. You can even try other ollama models like tinyllama, llama3.1 etc
2. Then we set the [embedding model](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/settings/)
   1. This model is used for calculating similarity and top-k retrieval
3. We also set the [chunk size](https://docs.llamaindex.ai/en/stable/optimizing/basic_strategies/basic_strategies/#chunk-sizes)
4. Also see [this post](https://www.llamaindex.ai/blog/evaluating-the-ideal-chunk-size-for-a-rag-system-using-llamaindex-6207e5d3fec5) about setting the right chunk size


In [7]:


base_model = 'llama3.2'

Settings.embed_model = HuggingFaceEmbedding(
    model_name='llmrails/ember-v1'
) # Refer

Settings.chunk_size = 2048


We also do `PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0`. This makes pytorch use all the available data. Without this, you may get an error: `MPS backend out of memory`

In [8]:
# For Mac and Linux
!export PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
# For Windows
# !set PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

### The below code instantiates a class that handles Qdrant DB used for building RAG
1. We first instantiate a QdrantClient that acts as a cliet to communicate with the Qdrant DB
2. In the `index_data` method, we create a colletion using `QdrantVectorStore`, this is designed for storing and retrieving vector embeddings
3. We then create a StorageContext that manages QdrantVectorStore
4. We use a `try..except` block, because the some documents can be bigger than our chunk size, in which case, we simply drop the document

### [QdrantVectorStore](https://docs.llamaindex.ai/en/stable/examples/vector_stores/QdrantIndexDemo/)
Purpose: It is a direct interface to the Qdrant vector database, which is designed for storing and retrieving vector embeddings.

Responsibilities:

Stores vector embeddings and associated metadata.
Manages interaction with the Qdrant database, including CRUD operations on collections, vectors, and payloads.
Facilitates vector similarity searches to retrieve the most relevant vectors based on a query embedding.

### [StorageContext](https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/)
Purpose: This is a higher-level abstraction used in frameworks like llama_index to manage various storage backends (including vector stores) seamlessly.

Responsibilities:

Acts as a bridge between data (e.g., documents, embeddings) and specific storage implementations (e.g., Qdrant, Weaviate, in-memory storage).
Provides a unified interface to access and manipulate data without being tied to a specific backend.
May combine data sources, such as documents in a database and vector embeddings, into a single context.


In [10]:
class QdrantHandler:
    def __init__(self, collection_name='finance_collection', host='localhost', port=6333):
        self.host = host
        self.port = port
        self.qdrant_client = QdrantClient(host=self.host, port=self.port) #Instantiate Qdrant DB with given port and host
        self.collection_name=collection_name #Name of the Qdrant collection
        
    def index_data(self, llama_documents):
        vector_store = QdrantVectorStore(client=self.qdrant_client, collection_name=self.collection_name)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        # print(llama_documents)
        index = VectorStoreIndex([]) 
        for doc in tqdm(llama_documents[:]):
            try:
                index.insert(doc)
            except (ValueError, RuntimeError) as v:
                print(f'Skipped: {v}')

    def get_qdrant_client(self):
        return self.qdrant_client

### The below code creates a class to extract information from news data
1. We extract relavant information from [US_Financial_News_Articles](https://drive.google.com/file/d/1m2FAtyA_NvWsZ_V3OhHl0JgTGkDGp6kp/view?usp=sharing)
2. This data contains several fields out of which we only fetch information such as location, title of the article, published date, news content
3. Other details such as author name, website name, article url etc are skipped, since they are not important for our task
4. With the extracted information we create a text embedding, and append the text and its embedding in the form of `Document`

In [13]:
# Class that handles pre-processing of US News data

class DataExtractionBaseClass(abc.ABC):
    def __init__(self, qdrant_client):
        self.qdrant_client = qdrant_client

    abc.abstractmethod
    def extract_data(self, *vargs) -> List[Document]:
        return 

class USNewsDataExtraction(DataExtractionBaseClass):
    def __init__(self, qdrant_client):
        super().__init__(qdrant_client)

    def get_locations(self, json_data):
        locations_list = json_data['entities']['locations']
        locations = []
        for ld in locations_list:
            if ('name' in ld) and ld['name'].strip()!='':
                locations.append(ld['name'])    
    
        if locations:
            return ', '.join(locations)
    
        return ''

    def json_2_text(self, json_data):
        title = json_data['title']
        location = self.get_locations(json_data)
        
        title_template = f'The title of the news article is {title}.'
        if location:
            location_template = f'The locations relavant to the article are: {location}.'
        else:
            location_template = ''
        published = json_data['published'][:10]
        date_template = f'This article was published on {published}'
        article = json_data['text']
        article_template = f'Article: {article}'
    
        item = {'title': title, 'location': location, 'published':  published, 'article': article}
        text = ' '.join([title_template, location_template, date_template, article_template])
        return text, item

    def extract_data(self, qdrant_client, number_of_folders=4, number_of_files_per_folder=10000, collection_name='finance_collection'):
        base_folder = 'data/US_Financial_News_Articles'
        c=0
        llama_documents = []
        for folder_name in os.listdir(base_folder)[:number_of_folders]:
            folder_path = os.path.join(base_folder, folder_name)
            if not os.path.isdir(folder_path):
                continue
            for file_name in tqdm(os.listdir(folder_path)[:number_of_files_per_folder]):
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                    text, item = json_2_text(json_data)
                    point_id = str(uuid.uuid4())
                    vector = Settings.embed_model.get_text_embedding(text)
                    point = {
                    "id": point_id,
                    "vector": vector,
                    "payload": {
                        "title": item["title"],
                        "location": item["location"],
                        "published": item["published"],
                        "article": item["article"],
                    }
                    }
                c+=1
    
                document = Document(metadata=item, text=text)
                llama_documents.append(document)
                # qdrant_client.upsert(collection_name=collection_name, points=[point])
        return llama_documents
   

Sometimes creating a collection when that collection name already exists might lead to unexpected results, its a good idea to delete collection if required

In [36]:
# Delete old collection if necessary
# client.delete_collection('finance_collection')

### Create an instance of Qdranthandler and get qdrant client

In [14]:


qdrant_handler = QdrantHandler()
qdrant_client = qdrant_handler.get_qdrant_client()

### Instantiate US News data extractor and create news - embedding pairs

In [214]:

data_extractor_us = USNewsDataExtraction(qdrant_client)
llama_documents = data_extractor_us.extract_data(qdrant_client, 4, 50000)

100%|███████████████████████████████████| 50000/50000 [2:43:11<00:00,  5.11it/s]


### Index the data; this will create a qdrant vector store later used for retrieval, as mentioned before some files that are too big are skipped


In [222]:

qdrant_handler.index_data(llama_documents)

  0%|                                    | 10/150000 [00:06<29:15:51,  1.42it/s]

Skipped: Metadata length (4630) is longer than chunk size (2048). Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


  0%|                                   | 102/150000 [00:56<22:37:04,  1.84it/s]

Skipped: Metadata length (7425) is longer than chunk size (2048). Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


  0%|                                   | 164/150000 [02:01<20:48:50,  2.00it/s]

...

Skipped: Metadata length (2548) is longer than chunk size (2048). Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


  0%|                                   | 366/150000 [04:16<16:19:10,  2.55it/s]

Skipped: Metadata length (2190) is longer than chunk size (2048). Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


  0%|                                   | 381/150000 [04:26<29:05:38,  1.43it/s]


KeyboardInterrupt: 

### The below creates a class to help in retrieval
1. We create a custom class to set a score_threhold, this decides the threshold / cosine similarity that we would like to set while retrieving the documents
2. Higher the threshold more the filtering
3. We also set how many matches to fetch using `similarity_top_k`


In [20]:
# Creates a Qdrant service - used by the model

class CustomQdrantClient:
    def __init__(self, client) -> None:
        self._client: QdrantClient = client

    def collection_exists(self, c):
        return self._client.collection_exists(c)

    def search(self, collection_name, query_vector, limit, query_filter):
        return self._client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=limit,
            query_filter=query_filter,
            score_threshold = 0.3
        )

class QdrantService:
    def __init__(self, client, collection_name='finance_collection'):
        self.collection_name = collection_name
        self.custom_qdrant_client = CustomQdrantClient(client)

    def get_vector_store_index(self):

        if not self.custom_qdrant_client.collection_exists(c=self.collection_name):
            self.logger.warning(f"Collection {self.collection_name} does not exist !")
            return None

        vector_store = QdrantVectorStore(
            client=self.custom_qdrant_client,
            collection_name=self.collection_name,
            parallel=1,
        )
        index: VectorStoreIndex = VectorStoreIndex.from_vector_store(vector_store=vector_store)
        query_engine: BaseQueryEngine = index.as_query_engine(
            similarity_top_k=5,
            verbose=False, streaming=False
        )
        return query_engine


#### Pydantic class - for data formatting

In [24]:


class Message(BaseModel):
    role: str
    content: str

#### Main class for QA
1. We create instances for Qdrant service (DB), Vector store and `historical_messages` which helps keep track of the historical chat up to 3 conversations
2. `get_model_instance` method fetches the instance of Ollama, `base_model_name` can only be a Ollama model like llama3, tinyllama etc
3.  For finetuned model, we create `peft_base_model` and its tokenizer `peft_tokenizer`
4.  We create a `prompt`, this is used for all messages to the model
5.  We then create the instance of Qdrant Service DB, and vector index
6.  Then using `peft_base_model` we create the Peft model
7.  The `llm_request` method takes the input query and model_type
8.  The model_type can ['Pure LLM', 'LLM + RAG', 'Finetuned LLM']
9.  The `llm_request` method when run on 'LLM + RAG', tries to use the RAG, if no suitable text in the documents is found for the given query, it falls back to running simple LLM

In [42]:

class FinanceQA:

    def __init__(self, qdrant_client, base_model_name='llama3.2', peft_base_model_name='google/flan-t5-base'):
        self.base_model_name = base_model_name
        self.base_model: CustomLLM = self.get_model_instance()
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.peft_base_model_name = peft_base_model_name
        self.historical_messages: List[Message] = []

        self.peft_base_model = AutoModelForSeq2SeqLM.from_pretrained(peft_base_model_name, torch_dtype=torch.bfloat16)
        self.peft_tokenizer = AutoTokenizer.from_pretrained(peft_base_model_name)
        # self.llm_model: CustomLLM = self.get_model_instance()
        self.prompt = 'You are Q&A assistant in Finance. Please provide a concise answer, up to 400 words. Focus on key points, avoid unnecessary details, and ensure important context is preserved.'
        self.set_defaults()



        self.qdrant_ser: QdrantService = QdrantService(qdrant_client)
        self.vector_store_index: BaseQueryEngine = self.qdrant_ser.get_vector_store_index()


        # PEFT
        self.peft_model = PeftModel.from_pretrained(self.peft_base_model,
                                               # './peft-dialogue-summary-checkpoint-local',
                                                    './peft-sent-model',
                                               torch_dtype=torch.bfloat16,
                                               is_trainable=False)  ## is_trainable mean just a forward pass jsut to get a sumamry


    def get_model_instance(self):
        model_instance = Ollama(
            # model = 'phi3:3.8b-mini-128k-instruct-q8_0',
            model=self.base_model_name,
            request_timeout=480,
            temperature=0.3,
            tokenizer_mode="slow",
            context_window=3000,
            additional_kwargs={
                'num_thread': 8,
                'num_ctx': 2500,
                'num_predict': 650,

            },
            base_url='http://localhost:11434')
        return model_instance


    def update_historical_context(self, message: Message) -> None:
        if len(self.historical_messages) == 8:
            # Remove 3rd and 4th element. that is, the 2nd question and answer pair
            del self.historical_messages[3]
            del self.historical_messages[3]

        self.historical_messages.append(message)

    def convert_to_format(self, messages):
        chat_messages = []
        for message in messages:
            chat_message = ChatMessage(
                role=message.role,  # Convert string to MessageRole enum
                content=message.content
            )
            chat_messages.append(chat_message)
        return chat_messages

    def clear(self):
        self.historical_messages.clear()

    def set_defaults(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        Settings.llm = self.base_model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name='llmrails/ember-v1', device=device
        )
        system_prompt_message = Message(content=self.prompt, role='system')
        self.historical_messages.append(system_prompt_message)

    def handle_finetuned(self, message: Message):
        prompt = self.prompt + '\n' + message.content
        input_ids = self.peft_tokenizer(prompt, return_tensors='pt').input_ids
        peft_model_outputs = self.peft_model.generate(input_ids=input_ids,
                                                 generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
        peft_model_text_output = self.peft_tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
        response = ChatResponse(
            message=ChatMessage(role=MessageRole.ASSISTANT, content=peft_model_text_output))
        return peft_model_text_output



    def llm_request(self, query: str, model_type):
        message = Message(role='user', content=query)

        if model_type == 'Finetuned LLM':
            return self.handle_finetuned(message)
        if model_type == 'LLM + RAG':
            use_qdrant = True
        else:
            use_qdrant = False
        self.update_historical_context(message)
        vector_response: RESPONSE_TYPE | None = None
        if use_qdrant and self.vector_store_index:
            vector_response = self.vector_store_index.query(self.historical_messages[-1].content)
            # self.logger.debug("vector index store response: {0} \n".format(vector_response))
            print('vector_response', vector_response)
        if (vector_response is None) or len(vector_response.source_nodes) <= 0:
            messages: List[ChatMessage] = self.convert_to_format(self.historical_messages)
            response_message: ChatResponse = self.base_model.chat(messages)
        else:
            response_message = ChatResponse(
                message=ChatMessage(role=MessageRole.ASSISTANT, content=vector_response.response))
        self.update_historical_context(Message(role=MessageRole.ASSISTANT, content=response_message.message.content))
        return response_message.message.content

    def get_all_outputs(self, query):
        model_types = ['Pure LLM', 'LLM + RAG', 'Finetuned LLM']
        for mt in model_types:
            output = self.llm_request(query, mt)
            self.clear()
            print(f'{mt}: {output}')
            print('###################################################################################')
        


In [38]:
finance_qa = FinanceQA(qdrant_client)

In [39]:



query = '''Give the sentiment of the following statement:

This transaction will also rationalize our pulp and paper industry related solutions .

Sentiment:'''

finance_qa.get_all_outputs(query)

Pure LLM: The sentiment of the statement is neutral/informative. The language used is objective and descriptive, providing a factual connection between the transaction and the company's pulp and paper industry-related solutions without expressing any emotion or opinion.
vector_response Neutral.
LLM + RAG: Neutral.
Finetuned LLM: positive


In [40]:
query = '''What was Trump's role in US finance?'''

finance_qa.get_all_outputs(query)

Pure LLM: Donald Trump, the 45th President of the United States, had a significant role in US finance before his presidency. Here are some key aspects of his involvement:

1. Real Estate Development: Trump built his business empire on real estate development, particularly in New York City. He developed and managed numerous high-profile properties, including the Trump Tower, Trump Plaza Hotel and Casino, and the Mar-a-Lago resort.
2. Atlantic City Casinos: In the 1990s, Trump expanded his business interests to Atlantic City, where he built several casinos, including Trump Taj Mahal, Trump Marina, and the Trump Plaza Hotel and Casino. The casinos were highly successful, but ultimately led to financial difficulties for Trump.
3. Trump Organization: As the chairman of the Trump Organization, a private company that manages his business interests, Trump was responsible for overseeing various aspects of his business empire, including real estate development, hospitality, and entertainment.
4.

####################################################################################################
## Finetuning LoRA
###################################################################################################

#### The below install maybe required if you are going to finetune it, if it works with the current environment its well and good. If it doesn't consider creating a new environment with the below requirements for finetuning




In [ ]:
!pip install transformers==4.27.2 \

!pip install torch==1.13.1 \

!pip install torchdata==0.5.1 \

!pip install loralib==0.1.1 \

!pip install peft==0.3.0 
!pip install evaluate


!pip install datasets==2.11.0 


In [312]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch 
import time 
import evaluate  ## for calculating rouge score
import pandas as pd
import numpy as np
import abc
from tqdm import tqdm
import os
from datasets import Dataset, DatasetDict

''

In [ ]:
!export CUDA_VISIBLE_DEVICES="0"

In [ ]:

class FinetuneDataHandler(abc.ABC):
    
    def __init__(self):
        pass

    abc.abstractmethod
    def extract_data():
        pass

class FinQA:
    def __init__(self):
        self.dataset = load_dataset("ibm/finqa")


    def create_context(self, example):
        # Combine pre_text, table, and post_text into a single string
        pre_text = example.get("pre_text", "")
        table = example.get("table", [])
        post_text = example.get("post_text", "")
        # print(table)
    
        # print(table)
        df = pd.DataFrame(table)
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        df = df.set_index(df.columns[0])
        # display(df)
        df = df[~df.index.duplicated(keep='first')]
        table_str = df.to_dict('index')
        # 
        # table1 = df.iloc[1:].to_dict('records')
    
        # print(table1)
        # table_str = "\n".join(
        #     [", ".join([f"{key}: {value}" for key, value in row.items()]) for row in table1]
        # )
        # print(table_str)
    
    
        # Convert table into a readable format (e.g., rows of data)
        # table_str = "\n".join(
        #     [", ".join([f"{key}: {value}" for key, value in row.items()]) for row in table]
        # )
        
        # Combine everything into a readable format
        # print(table_str)
        context = f"Pre-text: {pre_text}\nTable:\n{table_str}\nPost-text: {post_text}"
        return context
    
    def preprocess_function(self, example):
        # Create context from the available fields
        context = self.create_context(example)
        
        # Combine the context and the question
        input_text = f"{context}\nQuestion: {example['question']}\nAnswer:"
        
        # The answer is the target output
        output_text = example["answer"]
        
        return {"input": input_text, "output": output_text}

    def extract_dataset(self):
        processed_dataset = self.dataset.map(self.preprocess_function, remove_columns=self.dataset["train"].column_names)
        return processed_dataset


class FinSum:

    def __init__(self):
        self.data_path_1 = '/kaggle/input/finsum/temp'
        self.data_path_2 = '/kaggle/input/finsum'

    def extract_data(self):
        input_column = ['document']
        output_column = ['summary']
        all_datasets = []
        for fol in [self.data_path_1]:
            for f in tqdm(os.listdir(fol)):
                fp = os.path.join(fol, f)
                df = pd.read_csv(fp)
                all_datasets.append(df)
        
        all_datasets = pd.concat(all_datasets, axis=0).iloc[:20000]
        all_datasets['input'] = all_datasets['document']
        # all_datasets['input'] = all_datasets['input'].apply(lambda x: "Summarize the following text: " + x)
        all_datasets['output'] = all_datasets['summary']
        # all_datasets['output'] = all_datasets['output'].apply(lambda x: "Summary: " + x)
        del all_datasets['document']
        del all_datasets['summary']
        dataset = Dataset.from_pandas(all_datasets)

        # Split into train, validation, and test sets (optional)
        train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
        train_valid_split = train_test_split['train'].train_test_split(test_size=0.1, seed=42)
        
        # Create a DatasetDict
        dataset_dict = DatasetDict({
            "train": train_valid_split['train'],
            "validation": train_valid_split['test'],
            "test": train_test_split['test'],
        })
        return dataset_dict


class FinSent:

    def __init__(self):
        self.data_path_1 = '/kaggle/input/finance-sentiment/Sentences_50Agree.txt'

    def extract_data(self):
        all_datasets = pd.read_csv('/kaggle/input/finance-sentiment/Sentences_50Agree.txt', encoding= "ISO-8859-1", sep='.@', header=None)
        all_datasets.columns = ['input', 'output']
        # all_datasets = pd.concat(all_datasets, axis=0).iloc[:20000]
        # all_datasets['input'] = all_datasets['document']
        # # all_datasets['input'] = all_datasets['input'].apply(lambda x: "Summarize the following text: " + x)
        # all_datasets['output'] = all_datasets['summary']
        # # all_datasets['output'] = all_datasets['output'].apply(lambda x: "Summary: " + x)
        # del all_datasets['document']
        # del all_datasets['summary']
        dataset = Dataset.from_pandas(all_datasets)

        # Split into train, validation, and test sets (optional)
        train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
        train_valid_split = train_test_split['train'].train_test_split(test_size=0.1, seed=42)
        
        # Create a DatasetDict
        dataset_dict = DatasetDict({
            "train": train_valid_split['train'],
            "validation": train_valid_split['test'],
            "test": train_test_split['test'],
        })
        return dataset_dict

In [ ]:
fin_sent = FinSent()
finsent_dataset = fin_sent.extract_data()

In [ ]:
## for more information of model https://huggingface.co/google/flan-t5-base
model_name = 'google/flan-t5-base'

# bfloat16 mean we are using the small version of flan-t5
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f'trainable model parameters: {trainable_model_params}\n \
            all model parameters: {all_model_params} \n \
            percentage of trainable model parameters: {(trainable_model_params / all_model_params) * 100} %'


print(print_number_of_trainable_model_parameters(original_model))

In [ ]:
def tokeninze_function(example):
    start_prompt = 'Give the sentiment of the following statement: \n\n'
    end_prompt = '\n\nSentiment: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["input"]]
    example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, 
                                     return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['output'], padding='max_length', truncation=True, 
                                 return_tensors='pt').input_ids
    
    return example
    
# The Dataseta ctually contains 3 diff splits: train, validation, and test.
# The tokenize_function code is handling all data across all splits in batches
# tokenize_datasets = finsum_dataset.map(tokeninze_function, batched=True)
tokenize_datasets = finsent_dataset.map(tokeninze_function, batched=True)
# tokenize_datasets = tokenize_datasets.remove_columns(['id', 'topic', 'input',
#                                                      'output'])

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(r=32, #rank 32,
                         lora_alpha=32, ## LoRA Scaling factor 
                         target_modules=['q', 'v'], ## The modules(for example, attention blocks) to apply the LoRA update matrices.
                         lora_dropout = 0.3,
                         bias='none',
                         task_type=TaskType.SEQ_2_SEQ_LM ## flan-t5
)

## target_modules='q', This represents the value projection layer in the transformer model. The value projection layer transforms input tokens into value vectors,
# which are the actual values that are attended to based on the attention scores computed from query and key vectors.

## target_modules='v',This typically refers to the query projection layer in a transformer-based model. The query projection layer is responsible for transforming 
# input tokens into query vectors, which are used to attend to other tokens in the sequence during self-attention mechanism.

In [ ]:
original_model = original_model.to('cuda:0')

In [ ]:
peft_model = get_peft_model(original_model, lora_config)

print(print_number_of_trainable_model_parameters(peft_model))

In [ ]:
peft_model = peft_model.to('cuda:0')


In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'
## this is we are again back to the hugging face trainer module
# training_args = TrainingArguments(output_dir=output_dir,
#                                        evaluation_strategy="steps",
#                                        auto_find_batch_size=True,
#                                        learning_rate=1e-3,
#                                        num_train_epochs=1,
#                                        # logging_steps=1,
#                                        max_steps=5000,
#                                        weight_decay=0.01,
#                                        save_total_limit=0,
#                                        # per_device_train_batch_size=1,
#                                        label_names = ['labels'],
#                                        save_steps=500,
#                                         report_to='none', ## can be wandb, but we are reporint to noe
                                       
#                 )

training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="steps",
            learning_rate=2e-5,
            per_device_train_batch_size=1,
            num_train_epochs=1,
            weight_decay=0.01,
            logging_dir="./logs",
            save_total_limit=0,
            save_steps=500,
            # fp16=True,  # Use mixed precision for faster training
            # use_cpu=False,
            label_names = ['labels']
        )

## this is same except we are using PEFT model instead of regular

peft_trainer = Trainer(model=peft_model, 
                      args=training_args,
                      train_dataset=tokenize_datasets['train'],
                      eval_dataset=tokenize_datasets['validation']
                 )
peft_trainer.args._n_gpu = 1


peft_trainer.train()

peft_model_path = './peft-dialogue-summary-checkpoint-local'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:
from peft import PeftModel

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                      './peft-sent-model',
                                      torch_dtype=torch.bfloat16,
                                      is_trainable=False) ## is_trainable mean just a forward pass jsut to get a sumamry
peft_model = peft_model.to('cuda:0')
index = 200 ## randomly pick index
dialogue = tokenize_datasets['test'][index]['input']
human_baseline_summary = tokenize_datasets['test'][index]['output']
# dialogue = 'Finance department is important in a company. It is crucial to have a it in all companies. All employees must be aware of this. The finance department must have highly qualified people. It should operate within a company.'
prompt = f"""
Give the sentiment of the following statement:

{dialogue}

nSentiment:
"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids

input_ids = input_ids.to('cuda:0')
peft_model = peft_model.to('cuda:0')
original_model = original_model.to('cuda:0')
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(f'Human Baseline: \n{human_baseline_summary}\n')
print(f'Original Model Output \n{original_model_text_output}\n')
print(f'Peft Model Output \n{peft_model_text_output}\n')